This script is used to clean the database.

In [26]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = '/Users/yuanlu/Code/test/youtube-top-10000-channels/data/output-raw.db'
conn = sqlite3.connect(db_path)

# Print tables in database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in database:")
for table in tables:
    print(f"- {table[0]}")


Tables in database:
- combined_data
- unique_youtube_channel_urls


In [27]:
# Get size of combined_data table
cursor.execute("SELECT COUNT(*) FROM combined_data")
row_count = cursor.fetchone()[0]
print(f"Size of combined_data table: {row_count:,} rows")


Size of combined_data table: 1,176,396 rows


In [28]:
cursor.execute("SELECT COUNT(DISTINCT youtube_channel_url) FROM combined_data")
unique_count = cursor.fetchone()[0]
print(f"Number of unique youtube_channel_urls: {unique_count}")

Number of unique youtube_channel_urls: 42437


In [29]:
# Percentage of unique youtube_channel_urls
print(f"Percentage of unique youtube_channel_urls: {unique_count / row_count * 100:.2f}%")


Percentage of unique youtube_channel_urls: 3.61%


In [24]:
# Drop the unique_youtube_channel_urls table if it exists
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS unique_youtube_channel_urls")
    conn.commit()

In [25]:
# Create a new table with unique youtube_channel_urls, it should have the same columns as combined_data. Make sure youtube_channel_url is unique.
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE unique_youtube_channel_urls AS 
        SELECT *
        FROM combined_data
        GROUP BY youtube_channel_url
    """)
    conn.commit()

In [31]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = '/Users/yuanlu/Code/test/youtube-top-10000-channels/data/output-edit.db'
conn = sqlite3.connect(db_path)

# Print tables in database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in database:")
for table in tables:
    print(f"- {table[0]}")


Tables in database:
- combined_data
- unique_youtube_channel_urls


In [32]:
# delete combined_data table
cursor.execute("DROP TABLE IF EXISTS combined_data")
conn.commit()


In [38]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = '/Users/yuanlu/Code/test/youtube-top-10000-channels/data/output-edit.db'
conn = sqlite3.connect(db_path)

# Print tables in database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in database:")
for table in tables:
    print(f"- {table[0]}")


Tables in database:
- unique_youtube_channel_urls


In [39]:
# check if url_verified_status value is bad
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls WHERE url_verified_status = 'bad'")
bad_count = cursor.fetchone()[0]
print(f"Number of rows with bad url_verified_status: {bad_count}")


Number of rows with bad url_verified_status: 36314


In [40]:
# check if url_verified_status value is good
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls WHERE url_verified_status = 'good'")
good_count = cursor.fetchone()[0]
print(f"Number of rows with good url_verified_status: {good_count}")


Number of rows with good url_verified_status: 6123


In [42]:
# Get distinct accessibility_message values and their counts
cursor.execute("""
    SELECT accessibility_message, COUNT(*) as count 
    FROM unique_youtube_channel_urls 
    WHERE accessibility_message IS NOT NULL
    GROUP BY accessibility_message
    ORDER BY count DESC
""")
results = cursor.fetchall()

print("\nAccessibility message values and counts:")
for message, count in results:
    print(f"- {message}: {count}")




Accessibility message values and counts:
- HTTP 204: 29337
- Channel not found: 6863
- Accessible: 6119
- Request error: HTTPSConnectionPool(host='www.youtube.com', port=443): Max retries exceeded with url: /@nrkp3 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='www.youtube.com', port=443): Read timed out. (read timeout=3)")): 1
